In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
# import requests
import bs4
import json
import csv
import random
import json
from datetime import datetime
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
options = Options()
options.add_experimental_option("detach", True)##讓瀏覽器不要自動關閉
options.add_argument("--log-level=3")
options.add_argument('--user-agent=%s' % user_agent)
current_datetime = datetime.now().strftime("%Y%m%d_%H%M")


global firsttime 
firsttime = True
# webUrl=  "https://www.guidefitter.com/hunting/united-states?page=1"

chrome = webdriver.Chrome(executable_path='C:\\Users\Modern 14\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', chrome_options=options)##開啟瀏覽器



<ipython-input-26-505dcd373abf>:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome(executable_path='C:\\Users\Modern 14\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', chrome_options=options)##開啟瀏覽器
<ipython-input-26-505dcd373abf>:27: DeprecationWarning: use options instead of chrome_options
  chrome = webdriver.Chrome(executable_path='C:\\Users\Modern 14\\Desktop\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe', chrome_options=options)##開啟瀏覽器


In [27]:
def run_webdriver(webUrl):
    chrome.get(webUrl)##前往網址
    tripNameList=[]
    discriptionList = []
    linkList = []
    priceList = []
    locations = chrome.find_elements(By.CSS_SELECTOR,"[class='StyledFlex-LibWebComponents__sc-o6h6zm-0 ftSaxO']")##找到所有的地點
    # print(locations)
    try:
        for location in locations:
            tripName = location.find_element(By.CSS_SELECTOR,"[class*='gkORkG']").text##找到景點名稱
            mountainContents = location.find_elements(By.CSS_SELECTOR,"[class*='kBSMqm']")##找到景點下面的所有內容
            link =location.find_element(By.CSS_SELECTOR,"[class='StyledText-LibWebComponents__sc-1p4y2gp-0 kcPFV']").get_attribute("href")##找到景點的連結
            # print(mountain)
            for mountainContent in mountainContents:
                discription = mountainContent.find_element(By.CSS_SELECTOR,"[class='StyledText-LibWebComponents__sc-1p4y2gp-0 jQCrSX']").text##找到那座山相關景點的連結
                
                # try:
                #     price = mountainContent.find_element(By.CSS_SELECTOR,"[class='StyledText-LibWebComponents__sc-1p4y2gp-0 fBtTjq']").text     
                # except:
                #     price = mountainContent.find_element(By.XPATH,'//span[@class="StyledText-LibWebComponents__sc-1p4y2gp-0 fBtTjq"]').text
                try:
                    price = mountainContent.find_element(By.CSS_SELECTOR,"[class*='fBtTjq']").text     
                except:
                    price = None
                tripNameList.append(tripName)
                discriptionList.append(discription)
                linkList.append(link)
                priceList.append(price)
    except:
        print(tripName,link,"error")

    return tripNameList,discriptionList,linkList,priceList

In [28]:
def crawl_location(locationLink):
    try:
        chrome.get(locationLink)##前往網址
        time.sleep(random.randint(5, 9))
        WebDriverWait(chrome, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR,"[class*='gOppSE']"))
        )
        mountainLocation =chrome.find_element(By.CSS_SELECTOR,"[class*='gOppSE']").text
    except:
        mountainLocation = None
        print(locationLink,"Location 頁面錯誤")
    return mountainLocation

In [29]:
def save_csv(jsonData):
    global firsttime
    data_dict = jsonData
    file_name = "C:\\pythonpractice\\guidefitterProject\\data.csv"

    ##判斷是否為第一次執行，如果是第一次執行就寫入欄位名稱
    if firsttime == True:
        with open(file_name, mode='w', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(data_dict.keys())
            writer.writerow(data_dict.values())
            # print("第一次")
            firsttime= False
    else:
        with open(file_name, mode='a', newline='',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(data_dict.values())
            # print("第二次")
    ##########################################


In [30]:
if __name__ == '__main__':
    for a in range(13,119):
        webUrl=  f"https://www.guidefitter.com/hunting/united-states?page={a}"
        time.sleep(random.randint(5, 9))##等待網頁載入
        mountainName,mountainDiscription,mountainLinkList,mountainPrice = run_webdriver(webUrl)
        # print(mountainName[1], mountainDiscription[1], mountainLinkList[1], mountainPrice[1])
        print(f"第{a}頁")
        try:
            for i,moumtainLink in enumerate(mountainLinkList):
                mountainLocation = crawl_location(moumtainLink)
                # print(mountainLocation)
                data ={
                    "name":mountainName[i],
                    "discription":mountainDiscription[i],
                    "link":moumtainLink,
                    "price":mountainPrice[i],
                    "location":mountainLocation
                }
                try:
                    save_csv(data)
                except:
                    print("save_csv 有問題")
        except:
            print(mountainName[i],"error")
            continue
    # for i,moumtainLink in enumerate(mountainLinkList):
    #     mountainLocation = crawl_location(moumtainLink)
    #     # print(mountainLocation)
    #     data ={
    #         "name":mountainName[i],
    #         "discription":mountainDiscription[i],
    #         "link":moumtainLink,
    #         "price":mountainPrice[i],
    #         "location":mountainLocation
    #     }
    #     save_csv(data)

    ##########################################       
    print("完成全部")
    print(f"資料已存到 {'C:/pythonpractice/guidefitterProject/data.csv'}")

第13頁
第14頁
第15頁
第16頁
第17頁
第18頁
第19頁
第20頁
第21頁
第22頁
第23頁
第24頁
第25頁
第26頁
第27頁
第28頁
第29頁
第30頁
第31頁
第32頁
第33頁
第34頁
第35頁
第36頁
第37頁
第38頁
第39頁
第40頁
第41頁
第42頁
第43頁
第44頁
第45頁
第46頁
第47頁
第48頁
第49頁
第50頁
第51頁
第52頁
第53頁
第54頁
第55頁
第56頁
第57頁
第58頁
第59頁
第60頁
第61頁
第62頁
第63頁
第64頁
第65頁
第66頁
第67頁
第68頁
第69頁
第70頁
第71頁
第72頁
第73頁
第74頁
第75頁
第76頁
第77頁
第78頁
第79頁
第80頁
第81頁
第82頁
第83頁
第84頁
第85頁
第86頁
第87頁
第88頁
第89頁
第90頁
第91頁
第92頁
第93頁
第94頁
第95頁
https://www.guidefitter.com/Blackland-Outfitters Location 頁面錯誤
https://www.guidefitter.com/bullpeakoutfitters1 Location 頁面錯誤
第96頁
第97頁
第98頁
第99頁
第100頁
第101頁
第102頁
第103頁
第104頁
第105頁
第106頁
第107頁
第108頁
第109頁
第110頁
第111頁
第112頁
第113頁
第114頁
第115頁
第116頁
第117頁
第118頁
完成全部
資料已存到 C:/pythonpractice/guidefitterProject/data.csv
